## Remote crawler database (server)

### System initialization

In this tutorial we re-use PostgreSQL database Docker image we have presented in *Web crawling - basic tools* notebook. Prior to running the example, set up the database server.


### Database access logic

We use the same functions *increase_db_values* and *increase_db_values_locking* that can be accessed securely over the internet as in local example from *Web crawling - basic tools* notebook. We just ommit the increases parameter as client will call the method multiple times.

In [15]:
import threading
import psycopg2

lock = threading.Lock()

def reset_db_values():
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword", database="wier")
    conn.autocommit = True
    
    cur = conn.cursor()
    cur.execute("UPDATE showcase.counters SET value = 0")
    
    cur.close()
    conn.close()
    
def print_db_values():
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword", database="wier")
    conn.autocommit = True

    retVal = []
    print("\nValues in the database:")
    cur = conn.cursor()
    cur.execute("SELECT counter_id, value FROM showcase.counters ORDER BY counter_id")
    for counter_id, value in cur.fetchall():
        print(f"\tCounter id: {counter_id}, value: {value}")
        retVal.append({counter_id: value})
    cur.close()
    conn.close()
    return retVal

def increase_db_values(counter_id):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword", database="wier")
    conn.autocommit = True
    
    cur = conn.cursor()
    cur.execute("SELECT value FROM showcase.counters WHERE counter_id = %s", \
                (counter_id,))
    value = cur.fetchone()[0]
    cur.execute("UPDATE showcase.counters SET value = %s WHERE counter_id = %s", \
                (value+1, counter_id))
    cur.close()
    conn.close()
    
def increase_db_values_locking(counter_id):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword", database="wier")
    conn.autocommit = True

    with lock:
        cur = conn.cursor()
        cur.execute("SELECT value FROM showcase.counters WHERE counter_id = %s", \
                    (counter_id,))
        value = cur.fetchone()[0]
        cur.execute("UPDATE showcase.counters SET value = %s WHERE counter_id = %s", \
                    (value+1, counter_id))
        cur.close()
    conn.close()

### Server implementation

As it is not secure to open the database directly to the internet and you would like to have *centralized* control of the statements that are issued against the database we will implement a RESTful service to access the database. 

In [16]:
from flask import Flask
from flask import jsonify
from flask_httpauth import HTTPBasicAuth

app = Flask(__name__)

It is important to provide some authentication mechanism to protect your server endpoints so that only users with credentials are allowed to use them. 

We will just use a basic HTTP Authentication mechanism. The *verify_password* method will checke whether a user is in the database and return it as an object (just username in our case). If the user does not exist, the method returns *None.* The user object is then further accessible using method `basic_auth.current_user()`.

Methods we would like to protect need to be decorated using `@basic_auth.login_required`, otherwise *error_handler* method will be called.

In [17]:
basic_auth = HTTPBasicAuth()

@basic_auth.verify_password
def verify_password(username, password):
    if password == "BestNonencryptedPasswordEver!!!":
        return username
    else:
        return None

@basic_auth.error_handler
def basic_auth_error(status):
    return jsonify({"success": False, "message": "Wrong credentials!"}), status

It is nice to provide automatic mechanisms to return specific responses in case an error happens. Let's take care of 404 and 500 errors.

In [18]:
@app.errorhandler(404)
def not_found_error(error):
    return jsonify({"success": False, "message": "Object not found!"}), 404

@app.errorhandler(500)
def internal_error(error):
    # rollback db if using transactions
    return jsonify({"success": False, "message": "Server error!"}), 500

The most important part is to provide REST endpoints that will be accessible to the clients.

In [19]:
@app.route('/db/reset', methods=['POST'])
@basic_auth.login_required
def fl_restart():
    reset_db_values()
    return jsonify({"success": True})

@app.route('/db/get_values', methods=['GET'])
@basic_auth.login_required
def fl_get_values():
    retVal = print_db_values()
    return jsonify(retVal)
    
@app.route('/db/increase/<int:id>', methods=['POST'])
@basic_auth.login_required
def fl_inc_vals(id):
    increase_db_values(id)
    return jsonify({"success": True})

@app.route('/db/increase_locking/<int:id>', methods=['POST'])
@basic_auth.login_required
def fl_inc_vals_lock(id):
    increase_db_values_locking(id)
    return jsonify({"success": True})

Lastly we instantiate the server.

We should protect the server behind an SSL proxy or provide our own mechanisms to encrypt connections, otherwise anyone can intercept and check our messages. We use an automatically generated SSL certificate by the *Flask* library (good enough for the seminar work).

In [20]:
if __name__ == "__main__":
    app.run(ssl_context='adhoc')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on https://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Mar/2025 10:20:14] "GET /db/get_values HTTP/1.1" 200 -



Values in the database:
	Counter id: 1, value: 1194
	Counter id: 2, value: 1634


127.0.0.1 - - [03/Mar/2025 10:20:21] "GET /db/get_values HTTP/1.1" 200 -



Values in the database:
	Counter id: 1, value: 1194
	Counter id: 2, value: 1634


127.0.0.1 - - [03/Mar/2025 10:20:26] "POST /db/reset HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:26] "GET /db/get_values HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase/1 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase/1 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase_locking/2 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase/1 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase_locking/2 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase/1 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase_locking/2 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase/1 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase_locking/2 HTTP/1.1" 200 -



Values in the database:
	Counter id: 1, value: 0
	Counter id: 2, value: 0


127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase/1 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase_locking/2 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase/1 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase/1 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase_locking/2 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase/1 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase_locking/2 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase_locking/2 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase/1 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase/1 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase_locking/2 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase_locking/2 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 10:20:27] "POST /db/increase/1 HTTP/1


Values in the database:
	Counter id: 1, value: 2211
	Counter id: 2, value: 3000


Try to access the endpoint using your browser.